In [18]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

In [55]:



def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    #Let the page load. Change this number based on your internet speed.
    #Or, wait until the webpage is loaded, instead of hardcoding it.
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/deel-jobs-SRCH_KO0,4.htm'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    
    time.sleep(slp_time)

    try:
        for _ in range(33):  # Click the button 100 times
            load_more_button = driver.find_element_by_xpath("//button[@data-test='load-more']")
            load_more_button.click()
            time.sleep(4)  # Wait for more jobs to load
    except NoSuchElementException:
        print("No 'Show more jobs' button found.")
    '''
    try:
        driver.find_element_by_xpath('.//buton[@class="button_Button__MlD2g button-base_Button__knLaX"]').click()
    except NoSuchElementException:
        print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
    '''    
    print('Started')
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.



         
        #Going through each job in this page
        #job_buttons = driver.find_elements(By.CSS_SELECTOR,"li[class^='JobsList_jobListItem']")  #jl for Job Listing. These are the buttons we're going to click.
        job_buttons = driver.find_elements(By.CSS_SELECTOR,"li[class^='JobsList_jobListItem']")
        print(len(job_buttons))
        for job_button in job_buttons: 

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            if len(jobs) >= num_jobs:
                break
           

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False


          
            while not collected_successfully:
                try:
                    company_name = job_button.find_element_by_xpath('.//span[@class="EmployerProfile_compactEmployerName__LE242"]').text
                    location = job_button.find_element_by_xpath('.//div[@class="JobCard_location__rCz3x"]').text
                    job_title = job_button.find_element_by_xpath('.//a[contains(@class, "JobCard_jobTitle___7I6y")]').text
                    print(company_name)
                    try: 
                        driver.find_element_by_xpath(".//button[@class='JobDetails_showMore___Le6L']").click()
                    except:
                        print('Show button not foud')
                    time.sleep(1)

                    job_description = driver.find_element(By.XPATH, './/div[@class="JobDetails_jobDescription__uW_fK JobDetails_showHidden__C_FOA"]').text
                    collected_successfully = True

                except:

                    time.sleep(2)

            try:
                salary_estimate = job_button.find_element_by_xpath('.//div[@class="JobCard_salaryEstimate__arV5J"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = job_button.find_element_by_xpath('.//div[@class="EmployerProfile_ratingContainer__ul0Ef"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
            


            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))
                
            


            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            #try:
                #driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
                

            try:
                size = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Size']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Founded']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Type']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Industry']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Sector']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = driver.find_element(By.XPATH, "//div[@class='JobDetails_overviewItem__cAsry']//span[@class='JobDetails_overviewItemLabel__KjFln' and text()='Revenue']/following-sibling::div[@class='JobDetails_overviewItemValue__xn8EF']").text
            except NoSuchElementException:
                revenue = -1


            '''
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

            '''
                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
         
            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            


            #add job to jobs
             
                 
        #Clicking on the "next page" button
        
        
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [56]:
path = "E:\Projects\ds_salary_proj\chromedriver"
df = get_jobs('data scientist',899, False, path, 5)

No 'Show more jobs' button found.
Started
901
Progress: 0/899
Exacore
Progress: 1/899
Red Dog Media
Progress: 2/899
Sallie Mae
Progress: 3/899
Thrive Bioscience
Progress: 4/899
Senior Care Partners PACE
Progress: 5/899
D3 Air and Space Operations Inc
Progress: 6/899
New Jersey Courts
Show button not foud
New Jersey Courts
Progress: 7/899
Scale AI
Progress: 8/899
Solvus Global
Progress: 9/899
Deloitte
Progress: 10/899
Paradyme Management
Show button not foud
Paradyme Management
Progress: 11/899
Cardiovascular Institute of San Diego, Inc.
Progress: 12/899
HumanTouch LLC
Show button not foud
HumanTouch LLC
Progress: 13/899
OPTX
Progress: 14/899
Penske Truck Leasing and Logistics
Progress: 15/899
PRPDG
Progress: 16/899
Sonder Research X
Progress: 17/899
DATSURA
Progress: 18/899
HumanTouch LLC
Progress: 19/899
The Friedkin Group
Show button not foud
The Friedkin Group
Progress: 20/899
Sysdata Consulting, Inc
Progress: 21/899
Pennum Industries
Show button not foud
Pennum Industries
Progress:

Progress: 232/899
CACI
Progress: 233/899
Pyramid Consulting, Inc
Progress: 234/899
Charles River Laboratories
Progress: 235/899
Technosphere, Inc.
Progress: 236/899
B4CORP
Progress: 237/899
Rocket Pharmaceuticals
Progress: 238/899
Purpose Financial
Progress: 239/899
Infosys
Progress: 240/899
St. Jude Children's Research Hospital
Progress: 241/899
DataAnnotation
Progress: 242/899
Sanofi
Progress: 243/899
CGI Group, Inc.
Progress: 244/899
SteadyMD
Progress: 245/899
TechOp Solutions International
Progress: 246/899
Ardent Eagle Solutions
Progress: 247/899
Genmab
Progress: 248/899
Infosys
Progress: 249/899
Vanguard
Progress: 250/899
Vaco
Progress: 251/899
Kforce
Progress: 252/899
Northrop Grumman
Progress: 253/899
Capital One
Progress: 254/899
Kelly
Progress: 255/899
Capital One
Progress: 256/899
Quirch Foods
Progress: 257/899
Experis
Progress: 258/899
Santander Holdings USA Inc
Progress: 259/899
Peraton
Progress: 260/899
Amazon.com
Progress: 261/899
Peraton
Progress: 262/899
Boston Consult

Progress: 479/899
Outlier
Progress: 480/899
NVIDIA
Progress: 481/899
Air Products
Progress: 482/899
Accenture
Progress: 483/899
Edison Welding Institute Inc
Progress: 484/899
Northwestern Mutual
Progress: 485/899
CACI
Progress: 486/899
SteadyMD
Progress: 487/899
Howmet Aerospace
Progress: 488/899
Motion Recruitment
Progress: 489/899
Vaco
Progress: 490/899
Fidelity Investments
Progress: 491/899
Elevance Health
Progress: 492/899
Peraton
Progress: 493/899
Capital One
Progress: 494/899
Amazon.com
Progress: 495/899
AllSTEM Connections
Progress: 496/899
Santander Holdings USA Inc
Progress: 497/899
Peraton
Progress: 498/899
Motion Recruitment
Progress: 499/899
Motion Recruitment
Progress: 500/899
Vaco
Progress: 501/899
JPMorgan Chase & Co
Progress: 502/899
Core Group Resources
Progress: 503/899
Comerica Bank
Progress: 504/899
Everest Clinical Research
Progress: 505/899
Boston Consulting Group
Progress: 506/899
Guidehouse
Progress: 507/899
Revel IT
Progress: 508/899
Raytheon
Progress: 509/899


HCSC
Progress: 728/899
JPMorgan Chase & Co
Progress: 729/899
Peraton
Progress: 730/899
Kforce
Progress: 731/899
Capital One
Progress: 732/899
Equity Residential
Progress: 733/899
Booz Allen
Progress: 734/899
SOUTH COLLEGE
Progress: 735/899
Boston Consulting Group
Progress: 736/899
JPMorgan Chase & Co
Progress: 737/899
Boston Consulting Group
Progress: 738/899
Robert Half
Progress: 739/899
Boston Consulting Group
Progress: 740/899
JPMorgan Chase & Co
Progress: 741/899
Ursus, Inc.
Progress: 742/899
Outlier
Progress: 743/899
B4CORP
Progress: 744/899
Rokt
Progress: 745/899
Peraton
Progress: 746/899
Nestlé Purina Pet Care
Progress: 747/899
Kforce
Progress: 748/899
Charles River Laboratories
Progress: 749/899
Cherish Health
Progress: 750/899
St. Jude Children's Research Hospital
Progress: 751/899
Leidos
Progress: 752/899
Vagaro
Progress: 753/899
Capital One
Progress: 754/899
Sanofi
Progress: 755/899
Otsuka Pharmaceutical
Progress: 756/899
SteadyMD
Progress: 757/899
7-Eleven
Progress: 758/899

In [57]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,IT Tech II (Data Mining),USD 21.50 Per Hour (Employer est.),Are you passionate about computers and IT? We ...,-1,Exacore,"Iraan, TX",-1,-1,-1,-1,-1,-1
1,Senior Data Scientist,USD 200K (Employer est.),Title:\nSenior Data Scientist for Large Scale ...,4.5,Red Dog Media,"Henderson, NV",1 to 50 Employees,2008,Company - Private,Internet & Web Services,Information Technology,Unknown / Non-Applicable
2,"Manager, Model Validation",USD 76K - 126K (Glassdoor est.),"When you join Sallie Mae, you become a champio...",4.1,Sallie Mae,"Newark, DE",1001 to 5000 Employees,1973,Company - Public,Banking & Lending,Financial Services,$1 to $5 billion (USD)
3,Senior Biological Imaging Data Scientist,USD 150K - 175K (Employer est.),Title: Senior Biological Imaging Data Scientis...,5.0,Thrive Bioscience,"Beverly, MA",1 to 50 Employees,--,Company - Private,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,Unknown / Non-Applicable
4,Sr. Finance & Data Analyst,USD 64K - 92K (Glassdoor est.),Sr. Finance & Data Analyst\nWho is P.A.C.E?\nT...,3.4,Senior Care Partners PACE,"Battle Creek, MI",201 to 500 Employees,2008,Nonprofit Organization,Health Care Services & Hospitals,Healthcare,$5 to $25 million (USD)


In [58]:
df.to_csv('E:/Projects/ds_salary_proj/glassdoor_salary_new.csv', encoding='utf-8')